In [16]:
!pip install fasttext

In [17]:
import fasttext
import numpy as np
import pandas as pd
from keras_preprocessing import sequence
from keras_preprocessing.text import Tokenizer
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from tensorflow.keras.callbacks import EarlyStopping

In [18]:
df = pd.read_csv('tsd_train.csv')
df

,spans,text
0,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",Another violent and aggressive immigrant killi...
1,"[33, 34, 35, 36, 37, 38, 39]","I am 56 years old, I am not your fucking junio..."
2,"[0, 1, 2, 3]","Damn, a whole family. Sad indeed."
3,"[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]",What a knucklehead. How can anyone not know th...
4,"[32, 33, 34, 35, 36, 37, 38]","""who do you think should do the killing?""\n\nA..."
...,...,...
7934,"[8, 9, 10, 11]",Another fool pipes in.
7935,"[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 5...",So if a restaurant owner puts up a sign saying...
7936,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",Any faith that can't stand up to logic and rea...
7937,"[5, 6, 7, 8, 9, 10, 11]",This idiotic. Use the surplus to pay down the ...


In [19]:
import re
#предобработка текста
def list_replace(search, replacement, text):
    """
    Replaces all symbols of text which are present
    in the search string with the replacement string.
    """
    search = [el for el in search if el in text]
    for c in search:
        text = text.replace(c, replacement)
    return text

def clean_text(text):

    text = list_replace \
        ('\u00AB\u00BB\u2039\u203A\u201E\u201A\u201C\u201F\u2018\u201B\u201D\u2019', '\u0022', text)

    text = list_replace \
        ('\u2012\u2013\u2014\u2015\u203E\u0305\u00AF', '\u2003\u002D\u002D\u2003', text)

    text = list_replace('\u2010\u2011', '\u002D', text)

    text = list_replace \
            (
            '\u2000\u2001\u2002\u2004\u2005\u2006\u2007\u2008\u2009\u200A\u200B\u202F\u205F\u2060\u3000',
            '\u2002', text)

    text = re.sub('\u2003\u2003', '\u2003', text)
    text = re.sub('\t\t', '\t', text)

    text = list_replace \
            (
            '\u02CC\u0307\u0323\u2022\u2023\u2043\u204C\u204D\u2219\u25E6\u00B7\u00D7\u22C5\u2219\u2062',
            '.', text)

    text = list_replace('\u2217', '\u002A', text)

    text = list_replace('\u00C4', 'A', text)
    text = list_replace('\u00E4', 'a', text)
    text = list_replace('\u00CB', 'E', text)
    text = list_replace('\u00EB', 'e', text)
    text = list_replace('\u1E26', 'H', text)
    text = list_replace('\u1E27', 'h', text)
    text = list_replace('\u00CF', 'I', text)
    text = list_replace('\u00EF', 'i', text)
    text = list_replace('\u00D6', 'O', text)
    text = list_replace('\u00F6', 'o', text)
    text = list_replace('\u00DC', 'U', text)
    text = list_replace('\u00FC', 'u', text)
    text = list_replace('\u0178', 'Y', text)
    text = list_replace('\u00FF', 'y', text)
    text = list_replace('\u00DF', 's', text)
    text = list_replace('\u1E9E', 'S', text)
    # Removing punctuation
    text = list_replace(',.[]{}()=+-−*&^%$#@!~;:§/\|\?"\n', ' ', text)
    # Replacing all numbers with masks
    text = list_replace('0123456789', 'x', text)

    return text

In [20]:
#Переходим от символов к словам
X = [[]]
y = [[]]
for i in range(df.shape[0]):
    cl_text = clean_text(df.iloc[i,1])
    fl = np.zeros(len(cl_text))
    if(len(df.iloc[i,0]) > 2):
        arr = [int(a) for a in df.iloc[i,0][1:-1].split(', ')]
        fl[arr] = 1
    spl = cl_text.split(' ')
    prev_len = 0
    X_i = []
    y_i = []
    for j in range(len(spl)):
        if(len(spl[j]) > 0):
            X_i.append(spl[j])
            y_i.append(sum(fl[prev_len:prev_len+len(spl[j])])/len(spl[j]))
        prev_len = prev_len + len(spl[j]) + 1
    X.append(X_i)
    y.append(y_i)
X = X[1:]
y = y[1:]

In [21]:
cleaned_train_texts = [' '.join(i) for i in X]
maxlen = max([len(i) for i in X])
maxlen

201

In [13]:
! wget -O model_fasttext.bin https://www.dropbox.com/s/f8svib34687gyb4/rudrec_fasttext_model.bin?dl=0

--2020-11-08 11:49:09--  https://www.dropbox.com/s/f8svib34687gyb4/rudrec_fasttext_model.bin?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.1, 2620:100:601d:1::a27d:501
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/f8svib34687gyb4/rudrec_fasttext_model.bin [following]
--2020-11-08 11:49:09--  https://www.dropbox.com/s/raw/f8svib34687gyb4/rudrec_fasttext_model.bin
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb149ce8b5cf8d0ecde19706764.dl.dropboxusercontent.com/cd/0/inline/BCwk_wj4s1I4QgeXaDVDfyV5wG50sdhp__WfMvX-rHOBax_00vUhm2OiBwVC65Wl6fk00LSoxSLyDJOMxc5Lc0zCJXFzZwPKCN6sMFYuozfT4sxqk2OuNoP4YwWSRnGxcoQ/file# [following]
--2020-11-08 11:49:09--  https://ucb149ce8b5cf8d0ecde19706764.dl.dropboxusercontent.com/cd/0/inline/BCwk_wj4s1I4QgeXaDVDfyV5wG50sdhp__WfMvX-rHOBax_00vUhm2OiBwVC65Wl6fk00L

In [14]:
#Загружаем предобученную fasttext модель
fasttext_model_path = 'model_fasttext.bin'
fasttext_model = fasttext.load_model(fasttext_model_path)

In [22]:
#Получаем эмбеддинги
EMBEDDINGS_DIM = 200
tokenizer = Tokenizer(lower=True, char_level=False)
tokenizer.fit_on_texts(cleaned_train_texts)
word_seq_train = tokenizer.texts_to_sequences(cleaned_train_texts)
word_index = tokenizer.word_index

trash = []
for i in range(len(X)):
    if(len(X[i]) != len(word_seq_train[i])):
        trash.append(i)
for i in trash:
    word_seq_train.pop(i)
    X.pop(i)
    y.pop(i)
  
#Дополняем нулями до длины maxlen
word_seq_train = sequence.pad_sequences(word_seq_train, maxlen=maxlen, padding="post")

dictionary_size = len(word_index.keys())
embedding_matrix = np.zeros((dictionary_size + 1, EMBEDDINGS_DIM))\

for word, i in word_index.items():
    embedding_vector = fasttext_model.get_word_vector((word))
    if (embedding_vector is not None) and len(embedding_vector) > 0:
        embedding_matrix[i] = embedding_vector

In [23]:
#Дополняем нулями до длины maxlen
y = sequence.pad_sequences(y, maxlen=maxlen, padding="post")

In [24]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional

#Модель со слоем эмбеддингов, двунаправленным LSTM и полносвязным слоем к каждлму из выходов BiLSTM
input = Input(shape=(maxlen,))
out = Embedding(len(word_index.keys())+1, 200, input_length=maxlen, weights=[embedding_matrix])(input)
out = Dropout(0.1)(out)
out = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(out)
out = TimeDistributed(Dense(1, activation="sigmoid"))(out)
model = Model(input, out)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [25]:
#Разбиваем выборку на обучающую и тестовую и обучаем
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(word_seq_train, y, test_size=0.2)

In [26]:
model.fit(X_train, y_train, batch_size=32, epochs=3, validation_split=0.1, verbose=1)

Epoch 1/3
178/178 [==============================] - 161s 906ms/step - loss: 0.0812 - accuracy: 0.9835 - val_loss: 0.0399 - val_accuracy: 0.9877
Epoch 2/3
178/178 [==============================] - 161s 904ms/step - loss: 0.0419 - accuracy: 0.9874 - val_loss: 0.0363 - val_accuracy: 0.9889
Epoch 3/3
178/178 [==============================] - 164s 920ms/step - loss: 0.0391 - accuracy: 0.9879 - val_loss: 0.0367 - val_accuracy: 0.9893


In [34]:
pr = model.predict(X_test).reshape(y_test.shape)

In [28]:
import tensorflow as tf
np.mean(tf.keras.losses.binary_crossentropy(y_test, pr.reshape(y_test.shape)))

0.039707605

In [60]:
#Приводим выход к нужному формату для корректного подсчёта f1-score
import tensorflow as tf
arr1 = []
arr2 = []
for i in range(X_test.shape[0]):
    l = len(X_test[i])
    arr1.append([round(i) for i in list(pr[i][:l])])
    arr2.append(list(y_test[i][:l]))

In [61]:
from sklearn.metrics import accuracy_score, f1_score
np.mean([f1_score(arr2[i], arr1[i]) for i in range(len(arr2))])

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


0.588948730960844